In [1]:
import csv
from datetime import datetime
import os
import re
import requests
import twitter
import lxml.html

In [2]:
TWITTER_CONS_KEY = 'uhzOXpxETxaIXRKFmG4J10ztP'
TWITTER_CONS_SEC = 'SZBltYRGECF2pJ6t4PMqtRYRGReEFHVam2jxZa97xkJnyiSCki'
TWITTER_ACCESS_TOKEN = '1121670465838178304-4MmnwoM6bWS53X7hJwdij5Vsb09emC'
TWITTER_ACCESS_SEC = 'BEIZVVAyRNS3kkeeR37ZwyuZ0j2F1uxNV8dAQZTqlK9h2'

In [3]:

t = twitter.Api(
    consumer_key = TWITTER_CONS_KEY,
    consumer_secret = TWITTER_CONS_SEC,
    access_token_key = TWITTER_ACCESS_TOKEN, 
    access_token_secret = TWITTER_ACCESS_SEC,
    tweet_mode='extended' # this ensures that we get the full text of the users' original tweets
)

In [4]:
screen_name = "ZweHtet00836419"

In [16]:
# Call the Twitter API using the python-twitter library
first_200 = t.GetUserTimeline(screen_name=screen_name, count=200)

In [17]:
print(t.VerifyCredentials())

{"created_at": "Fri Apr 26 07:00:53 +0000 2019", "default_profile": true, "friends_count": 10, "id": 1121670465838178304, "id_str": "1121670465838178304", "name": "Zwe Htet Paing", "profile_background_color": "F5F8FA", "profile_image_url": "http://pbs.twimg.com/profile_images/1175429557777649664/iKTy6j2Z_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/1175429557777649664/iKTy6j2Z_normal.jpg", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "ZweHtet00836419", "status": {"created_at": "Fri Sep 20 12:03:36 +0000 2019", "full_text": "Teamwork making the dream work in the UEFA Champions League... Some things never change. \ud83e\udd29 But which was your fav\u2026 https://t.co/ZgN0yCkXlr", "id": 1175017663644364806, "id_str": "1175017663644364806", "lang": "en", "source": "<a href=\"https://mysite.com\" rel=\"nofo

In [18]:
print("There are %d tweets in our dataset, which has the type %s." % (len(first_200), type(first_200)))
print()




There are 3 tweets in our dataset, which has the type <class 'list'>.



In [19]:
print("Here's a sample of what the data from one tweet looks like:")
print(first_200[0])
print()

Here's a sample of what the data from one tweet looks like:
{"created_at": "Fri Sep 20 12:03:36 +0000 2019", "full_text": "Teamwork making the dream work in the UEFA Champions League... Some things never change. \ud83e\udd29 But which was your fav\u2026 https://t.co/ZgN0yCkXlr", "hashtags": [], "id": 1175017663644364806, "id_str": "1175017663644364806", "lang": "en", "source": "<a href=\"https://mysite.com\" rel=\"nofollow\">Personality Prediction using SVM</a>", "urls": [{"expanded_url": "https://twitter.com/i/web/status/1175005610762473472", "url": "https://t.co/ZgN0yCkXlr"}], "user": {"created_at": "Fri Apr 26 07:00:53 +0000 2019", "default_profile": true, "friends_count": 10, "id": 1121670465838178304, "id_str": "1121670465838178304", "name": "Zwe Htet Paing", "profile_background_color": "F5F8FA", "profile_image_url": "http://pbs.twimg.com/profile_images/1175429557777649664/iKTy6j2Z_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/1175429557777649664/iK

In [8]:
print("Each tweet is stored in a %s data structure." % type(first_200[0]))

Each tweet is stored in a <class 'twitter.models.Status'> data structure.


In [9]:
def get_tweets(first_200, screen_name, last_id):
    all_tweets = []
    all_tweets.extend(first_200)
    for i in range(900):
        new = t.GetUserTimeline(screen_name=screen_name, max_id=last_id-1)
        if len(new) > 0:
            all_tweets.extend(new)
            last_id = new[-1].id
        else:
            break
    
    return all_tweets

In [10]:
all_tweets = get_tweets(first_200, screen_name, first_200[-1].id)


In [11]:
print("There are %d tweets stored in a list as the all_tweets variable." % len(all_tweets))
print("The most recent tweet in our collection was sent %s and the oldest tweet was sent %s." % (
                                                                            all_tweets[0].created_at, 
                                                                            all_tweets[-1].created_at)
     )

There are 3 tweets stored in a list as the all_tweets variable.
The most recent tweet in our collection was sent Fri Sep 20 12:03:36 +0000 2019 and the oldest tweet was sent Fri Sep 20 12:02:28 +0000 2019.


In [20]:
print(all_tweets[0])

{"created_at": "Fri Sep 20 12:03:36 +0000 2019", "full_text": "Teamwork making the dream work in the UEFA Champions League... Some things never change. \ud83e\udd29 But which was your fav\u2026 https://t.co/ZgN0yCkXlr", "hashtags": [], "id": 1175017663644364806, "id_str": "1175017663644364806", "lang": "en", "source": "<a href=\"https://mysite.com\" rel=\"nofollow\">Personality Prediction using SVM</a>", "urls": [{"expanded_url": "https://twitter.com/i/web/status/1175005610762473472", "url": "https://t.co/ZgN0yCkXlr"}], "user": {"created_at": "Fri Apr 26 07:00:53 +0000 2019", "default_profile": true, "friends_count": 10, "id": 1121670465838178304, "id_str": "1121670465838178304", "name": "Zwe Htet Paing", "profile_background_color": "F5F8FA", "profile_image_url": "http://pbs.twimg.com/profile_images/1175429557777649664/iKTy6j2Z_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/1175429557777649664/iKTy6j2Z_normal.jpg", "profile_link_color": "1DA1F2", "profile

In [21]:
print("Data in the created_at attribute looks like this:", all_tweets[1].created_at)
print("Data in the hashtags attribute looks like this:", all_tweets[1].hashtags)
print("Data in the urls attribute looks like this:", all_tweets[1].urls)
print("Data in the source attribute looks like this:", all_tweets[1].source)

Data in the created_at attribute looks like this: Fri Sep 20 12:03:14 +0000 2019
Data in the hashtags attribute looks like this: []
Data in the urls attribute looks like this: []
Data in the source attribute looks like this: <a href="https://mysite.com" rel="nofollow">Personality Prediction using SVM</a>


In [14]:
def clean_hashtags(hashtags):
    """
    Turns data with any number of hashtags like this - [Hashtag(Text='STEMonStation')] - to a list like this -
    ['STEMonStation']
    """
    cleaned = []
    if len(hashtags) >= 1:
        for i in range(len(hashtags)):
            cleaned.append(hashtags[i].text)        
    return cleaned

def clean_urls(urls):
    """
    Turns data with any number of expanded urls like this - 
    [URL(URL=https://t.co/sYCFHKxzBf, ExpandedURL=https://youtu.be/34bFgA3H3hQ)]- to a list like this - 
    ["https://youtu.be/34bFgA3H3hQ"]
    """
    cleaned = []
    if len(urls) >= 1:
        for i in range(len(urls)):
            cleaned.append(urls[i].expanded_url)
    return(cleaned)
        

def clean_source(source):
    """
    Turns data including the source and some html like this - 
    <a href="https://www.sprinklr.com" rel="nofollow">Sprinklr</a> - to a list like this -
    ['Sprinklr']
    """
    raw = lxml.html.document_fromstring(source)
    return raw.cssselect('body')[0].text_content()


def string_to_datetime(date_str):
    """
    Turns a string including date and time like this - Sun Jul 01 21:06:07 +0000 2018 - to a Python datetime object
    like this - datetime.datetime(2018, 7, 1, 21, 6, 7, tzinfo=datetime.timezone.utc)
    """
    return datetime.strptime(date_str, '%a %b %d %H:%M:%S %z %Y')


In [19]:
def write_to_csv(tweets, filename):
    # the headers are the fields that we identified in step 4
    #headers = ['id', 'full_text', 'hashtags', 'urls', 'created_at', 'favorite_count', 'retweet_count', 'source']
    headers = ['id','full_text','source']
    
    # here we create the file and write the header row with the headers list
    # note that the 'filename' argument will be the name of the csv file
    with open(filename + '.csv', 'w', newline='',encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(headers)
        
        # in this loop, we write a new row for each tweet object, with the data taken from the tweet object in 
        # the order we listed the headers
        # note where we call the helper functions from step 4 on hashtags, urls, and source
        for item in tweets:
            writer.writerow([item.id, 
                             item.full_text, 
                             #clean_hashtags(item.hashtags), 
                             #clean_urls(item.urls), 
                             #item.created_at, 
                             #item.favorite_count, 
                             #item.retweet_count, 
                             clean_source(item.source)])
    csvfile.close()

In [22]:
def test_data(tweets):
    X=[]
    for item in tweets:
        X.append(item.full_text)
    return X

X = test_data(first_200)
Y = test_data(all_tweets)

In [24]:
print(X)

['Teamwork making the dream work in the UEFA Champions League... Some things never change. 🤩 But which was your fav… https://t.co/ZgN0yCkXlr', '😍 An elegant artist with a touch of genius... 🇧🇬 Happy retirement, Dimitar Berbatov', 'hello my name is Zwe Htet Paing and I love betting football match.']


In [25]:
print(Y)

['Teamwork making the dream work in the UEFA Champions League... Some things never change. 🤩 But which was your fav… https://t.co/ZgN0yCkXlr', '😍 An elegant artist with a touch of genius... 🇧🇬 Happy retirement, Dimitar Berbatov', 'hello my name is Zwe Htet Paing and I love betting football match.']


In [25]:
write_to_csv(first_200, screen_name + '_tweets')

In [33]:
import pandas as pd
df = pd.read_csv(screen_name + '_tweets.csv')

In [36]:
filename =screen_name + '_tweets.csv'
print(filename)

ZweHtet00836419_tweets.csv


In [ ]:
def create_dict(tweets):
    dict = {}
    for item in tweets:
        clean_source(item.source)
        dict[item.id_str] = {
            'full_text': item.full_text,
            #'hashtags': clean_hashtags(item.hashtags),
            #'urls': clean_urls(item.urls),
            #'created_at': string_to_datetime(item.created_at),
            #'favorite_count': item.favorite_count,
            #'retweet_count' : item.retweet_count,
            'source': clean_source(item.source)
        }
    return dict

In [ ]:
tweet_dict = create_dict(all_tweets)

In [ ]:
tweet_dict['1172908785582497792']

In [ ]:
print(tweet_dict)

In [ ]:
0.4955237907970503*3